In [157]:
#解析生成json的内容
#解析决策过程的日志
import os
import json
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def paese_result_json(log_path):
    #读取json文件
    res=json.load(open(log_path,"r"))['update']
    #去除掉init_state部分
    static={}
    for r in res:
        actions,states,tick=r['action'],r['state'],r['tick']
        #print(r['tick'])
        if r['tick']>148 and r['tick']<200:
            print(actions,states,tick)
        
        for action in actions:
            if str(action['class']) + '_' + str(action['id']) not in static:
                static[str(action['class']) + '_' + str(action['id'])]={}
            if action['action_type']=='SKILL_79':
                print(r['tick'])
            static[str(action['class']) + '_' + str(action['id'])][action['action_type']]=static[str(action['class']) + '_' + str(action['id'])][action['action_type']]+1 if action['action_type'] in static[str(action['class']) + '_' + str(action['id'])] else 1
            if 'damage' not in static[str(action['class']) + '_' + str(action['id'])]:
                static[str(action['class']) + '_' + str(action['id'])]['damage']=[]
            #print(action['class'] + '_' + str(action['id']),static[str(action['class']) + '_' + str(action['id'])]['damage'])
                #print(action)
            if 'damage' in action:
                for damage in action['damage']:
                    if type(damage[2])!=list:
                        damage[2]=[damage[2]]
                    static[str(action['class']) + '_' + str(action['id'])]['damage'].append(damage[2][0])                         
                    #['ATK', ['monster', 616666], [108], [108], ['Equal'], ['Default']]  [108] 是伤害值
            if 'effects' not in static[str(action['class']) + '_' + str(action['id'])]:
                static[str(action['class']) + '_' + str(action['id'])]['effects']=[]

                        
#             if action['class'] + '_' + str(action['id'])=='monster_716666':
#                 print('77777', action['class'] + '_' + str(action['id']))
            
                
            if 'effects' in action:
                if len(action['effects'])>0:
                    pass
#                     for r in action['effects']:
#                         static[str(action['class']) + '_' + str(action['id'])]['effects'].append(r[1])

        for state in states:
            if len(state)>0:
                for change in state:
                    if change[1][2]=="Hp":
                        #pass
                        if str(change[1][0]) + '_' + str(change[1][1]) not in static:
                            static[str(change[1][0]) + '_' + str(change[1][1])]={}
                        
                        if '最终HP' not in static[str(change[1][0]) + '_' + str(change[1][1])]:
                            static[str(change[1][0]) + '_' + str(change[1][1])]['最终HP']=''
                        static[str(change[1][0]) + '_' + str(change[1][1])]['最终HP']=change[2][1]

    for k in static.keys():
        if 'damage' not in static[k]:
            static[k]['damage']=[]
        if 'effects' not in static[k]:
            static[k]['effects']=[]
        #print(k,static[k])
        static[k]['总伤害']=sum(static[k]['damage'])
        static[k]['总伤害次数']=len(static[k]['damage'])
        static[k]['平均伤害']=sum(static[k]['damage'])/len(static[k]['damage']) if len(static[k]['damage'])>0 else 0
        static[k]['各类效果']= list(set(num for sublist in static[k]['effects'] for num in sublist))
        if 'RIGHT' not in static[k]:
            static[k]['RIGHT']=0
        if 'LEFT' not in static[k]:
            static[k]['LEFT']=0
        if 'TOP' not in static[k]:
            static[k]['TOP']=0
        if 'BOTTOM' not in static[k]:
            static[k]['BOTTOM']=0
        static[k]['总移动次数']=static[k]['RIGHT']+static[k]['LEFT']+static[k]['TOP']+static[k]['BOTTOM']
        
        
        #隐藏具体伤害
        static[k].pop('damage')
        static[k].pop('effects')

        #按照key 排序
        static[k]=dict(sorted(static[k].items(),key=lambda x:x[0]))
    print('总tick',res[-1]['tick'])
    return static


In [158]:
log_files_path = os.getcwd()+"/test_data"
    #读取这个路径下所有文件
log_files = os.listdir(log_files_path)
for log_file in log_files:
    if log_file.endswith(".json"):
        log_path = log_files_path+"/"+log_file
        static_res_json=paese_result_json(log_path)
        df=pd.DataFrame(static_res_json).T
        display(df)

72
144
144
[{'action_type': 'MOVE_START', 'move_start': [[4, 1, 6], [3, 1, 6], [5, 1, 6], [3, 1, 5], [5, 1, 5], [6, 1, 4], [7, 1, 5], [4, 1, 5], [5, 1, 4], [6, 1, 5], [5, 1, 3]], 'sequence': [{'id': 616666, 'speed': 176, 'class': 'monster'}, {'id': 1316666, 'speed': 154, 'class': 'monster'}, {'id': 7115666, 'speed': 154, 'class': 'monster'}, {'id': 8115666, 'speed': 154, 'class': 'monster'}, {'id': 16714666, 'speed': 154, 'class': 'monster'}, {'id': 5001, 'speed': 165, 'class': 'hero'}, {'id': 5011, 'speed': 165, 'class': 'hero'}, {'id': 5003, 'speed': 168, 'class': 'hero'}, {'id': 5002, 'speed': 180, 'class': 'hero'}, {'id': 5012, 'speed': 180, 'class': 'hero'}, {'id': 516666, 'speed': 185, 'class': 'monster'}, {'id': 716666, 'speed': 185, 'class': 'monster'}, {'id': 1216666, 'speed': 185, 'class': 'monster'}, {'id': 1416666, 'speed': 185, 'class': 'monster'}, {'id': 6115666, 'speed': 185, 'class': 'monster'}, {'id': 13712666, 'speed': 185, 'class': 'monster'}, {'id': 13713666, 'speed

,BOTTOM,LEFT,MOVE_START,RIGHT,SKILL_133,SKILL_134,TOP,WAIT,各类效果,平均伤害,总伤害,总伤害次数,总移动次数,最终HP,SKILL_135,SKILL_136,SKILL_130,SKILL_131,SKILL_132,EFFECT_64,EFFECT_87,SKILL_100,SKILL_98,SKILL_99,SKILL_88,SKILL_90,SKILL_137,SKILL_138,SKILL_77,SKILL_79,SKILL_80,SKILL_82,SKILL_78
monster_616666,6,11,16,10,8,2,2,2,[],65.9,659,10,29,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_1316666,4,4,30,8,4,2,3,23,[],72.0,432,6,19,501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_7115666,4,1,30,5,NaN,2,0,25,[],28.5,57,2,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_8115666,7,2,30,6,NaN,NaN,6,19,[],13.666667,82,6,21,402,4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_16714666,3,10,30,5,NaN,NaN,4,22,[],53.111111,478,9,22,495,NaN,NaN,4,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hero_5011,0,0,6,13,NaN,NaN,2,NaN,[],30.523263,579.942,19,15,0,NaN,NaN,NaN,NaN,NaN,6,6,1,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hero_5003,0,0,6,10,NaN,NaN,7,NaN,[],63.0,630,10,17,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
monster_516666,4,4,7,3,NaN,NaN,4,NaN,[],34.0,102,3,15,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN
monster_716666,5,5,9,6,NaN,NaN,3,1,[],30.25,121,4,19,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,NaN,NaN,NaN,NaN,NaN
hero_5001,0,0,4,3,NaN,NaN,3,NaN,[],67.777778,610.0,9,6,0,NaN,NaN,NaN,NaN,NaN,4,NaN,1,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
def parse_result_json_detail(log_path):
    res = json.load(open(log_path, "r"))['update']
    # 去除掉init_state部分
    static = {}
    for r in res:
        actions,states,tick=r['action'],r['state'],r['tick']
        for i in range(len(actions)):
            if actions[i]['id']==5001:
                if actions[i]['action_type']!='MOVE_START' and actions[i]['action_type']!='WAIT':
                    print(actions[i],states[i],r['tick'])
log_files_path = os.getcwd() + "/test_data"
    # 读取这个路径下所有文件
log_path = log_files_path + "/for_qiangye_skill_"+str(99)+".json"
log_path = log_files_path + "/result.json"

parse_result_json_detail(log_path)

{'action_type': 'TOP', 'move_position': [3, 1, 4], 'id': 5001, 'class': 'hero'} [['change', ['map', 0, 1, 'Block'], [2, 0]], ['change', ['map', 0, 2, 'Block'], [0, 2]], ['change', ['hero', 5001, 'position', 2], [3, 4]]] 33
{'action_type': 'TOP', 'move_position': [3, 1, 5], 'id': 5001, 'class': 'hero'} [['change', ['map', 0, 2, 'Block'], [2, 0]], ['change', ['map', 0, 3, 'Block'], [0, 2]], ['change', ['hero', 5001, 'position', 2], [4, 5]]] 33
{'action_type': 'TOP', 'move_position': [3, 1, 6], 'id': 5001, 'class': 'hero'} [['change', ['map', 0, 3, 'Block'], [2, 0]], ['change', ['map', 0, 4, 'Block'], [0, 2]], ['change', ['hero', 5001, 'position', 2], [5, 6]]] 33
{'action_type': 'SKILL_100', 'atk_range': [[5, 1, 6], [6, 1, 5], [7, 1, 6], [6, 1, 6]], 'atk_position': [6, 1, 6], 'release_range': [[3, 1, 3], [3, 1, 4], [3, 1, 5], [3, 1, 7], [3, 1, 8], [3, 1, 9], [4, 1, 4], [4, 1, 5], [4, 1, 6], [4, 1, 7], [4, 1, 8], [5, 1, 5], [5, 1, 6], [6, 1, 6], [3, 1, 6]], 'damage': [['ATK', ['monster', 5

In [187]:
def parse_decision_log(log_path):
    res=json.load(open(log_path,"r"))
    for k,v in res.items():
        if 'stepname' in v:
            if 'hero' in v:
                if v['hero']==516666:
                    if 'evaluate' in v['stepname'] or '决策树' in v['stepname']:
                        if  v['stepname']=='决策树-首次查找最终选择' or  v['stepname']=='决策树-多次查找最终选择':
                            #if '判断是否满足恢复判定' in v['selection'] and v['result_len']=='True':
                                print(k,v)
                        
def static_decision_log(log_path):
    static={}
    res=json.load(open(log_path,"r"))
    for k,v in res.items():
        if 'stepname' in v:
            if 'evaluate' in v['stepname'] or '决策树' in v['stepname']:
                if  v['stepname']=='决策树-首次查找最终选择' and v['result_len']!=0:
                    if v['hero'] not in static:
                        static[v['hero']]={}
                    if v['action'] not in static[v['hero']]:
                        static[v['hero']][v['action']]=1
                    static[v['hero']][v['action']]=static[v['hero']][v['action']]+1
                    
                if  v['stepname']=='决策树-多次查找最终选择':
                    if v['hero'] not in static:
                        static[v['hero']]={}
                    if v['action'] not in static[v['hero']]:
                        static[v['hero']][v['action']]=1
                    static[v['hero']][v['action']]=static[v['hero']][v['action']]+1
                if v['stepname']=='决策树-偏好决策':
                    if v['hero'] not in static:
                        static[v['hero']]={}
                    if v['action'] not in static[v['hero']]:
                        static[v['hero']]['偏好_'+str(v['action'])]=1
                    static[v['hero']]['偏好_'+str(v['action'])]=static[v['hero']]['偏好_'+str(v['action'])]+1

    df=pd.DataFrame(static).T
    display(df)

In [188]:
#统计情况
static_decision_log(os.getcwd()+"/../log/d0c112f87d014e9695ce495a2ab7a0212024_11_18_15_47_53logs.json")


,偏好_等待,移动到敌人附近并攻击,移动到敌人附近,等待,偏好_p_0_action,寻找治疗目标并治疗,移动到boss
616666,2.0,16.0,3.0,NaN,NaN,NaN,NaN
1316666,17.0,13.0,NaN,5.0,NaN,NaN,NaN
7115666,17.0,NaN,NaN,17.0,NaN,NaN,NaN
8115666,17.0,NaN,NaN,17.0,NaN,NaN,NaN
16714666,17.0,NaN,NaN,17.0,NaN,NaN,NaN
5003,NaN,NaN,NaN,NaN,2.0,NaN,NaN
5001,2.0,2.0,NaN,NaN,NaN,3.0,NaN
5011,2.0,2.0,NaN,NaN,NaN,4.0,NaN
5002,2.0,3.0,NaN,NaN,NaN,5.0,NaN
5012,2.0,6.0,NaN,NaN,NaN,5.0,2.0


In [189]:

tmp_log_file = os.getcwd()+"/../log/d0c112f87d014e9695ce495a2ab7a0212024_11_18_15_47_53logs.json"
parse_decision_log(tmp_log_file)

2024-11-18 15:47:59.325746_180 {'stepname': '决策树-首次查找最终选择', 'result_len': 2, 'action': '移动到敌人附近并攻击', 'hero': 516666}
2024-11-18 15:48:13.455329_529 {'stepname': '决策树-首次查找最终选择', 'result_len': 4, 'action': '移动到敌人附近并攻击', 'hero': 516666}
2024-11-18 15:48:37.881946_896 {'stepname': '决策树-首次查找最终选择', 'result_len': 1, 'action': '移动到敌人附近并攻击', 'hero': 516666}
2024-11-18 15:49:00.524512_1163 {'stepname': '决策树-首次查找最终选择', 'result_len': 1, 'action': '移动到敌人附近并攻击', 'hero': 516666}
2024-11-18 15:49:32.677429_1485 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '移动到敌人附近并攻击', 'hero': 516666}
2024-11-18 15:50:15.060729_1822 {'stepname': '决策树-首次查找最终选择', 'result_len': 3, 'action': '移动到敌人附近并攻击', 'hero': 516666}
2024-11-18 15:50:43.920183_2040 {'stepname': '决策树-首次查找最终选择', 'result_len': 1, 'action': '等待', 'hero': 516666}
2024-11-18 15:51:24.801754_2324 {'stepname': '决策树-首次查找最终选择', 'result_len': 1, 'action': '等待', 'hero': 516666}
2024-11-18 15:52:11.468304_2614 {'stepname': '决策树-首次查找最终选择', 'result_len': 

In [156]:
2024-11-18 11:46:15.819429_1552 {'stepname': '决策树-首次查找最终选择', 'result_len': 1, 'action': '等待', 'hero': 716666}


SyntaxError: invalid syntax (1450675318.py, line 1)